# COVID-19 Epidimiological Parameters

Lets begin by uploading the LSHTM R package epiparameter.

In [ ]:
# To use the R Magic commands
import sys
import os
conda_env_path = sys.executable.replace("/bin/python","")
os.environ["R_HOME"] = f"{conda_env_path}/lib/R"
%load_ext rpy2.ipython

In [ ]:
%%R -o covid_epi_params
#install.packages("epiparameter")
library(epiparameter)
db <- epiparameter_db()
covid_epi_params <- parameter_tbl(multi_epiparameter = db, disease = "COVID-19")

In [ ]:
covid_epi_params


In [ ]:
covid_epi_params.epi_distribution.value_counts()

## COVID-19 Infection Period/Rate of Becoming Uninfectious

Epiparameter seems to have nothing on the entire infection period. We will use the Killingley et al (2022) human challenge of anncestral COVID-19:
 
Killingley, B., Mann, A. J., Kalinova, M., Boyers, A., Goonawardane, N., Zhou, J., Lindsell, K., Hare, S. S., Brown, J., Frise, R. R., Smith, E., Hopkins, C., Noulin, N., Löndt, B., Wilkinson, T., Harden, S., McShane, H., Baillet, M., Gilbert, A., … Catchpole, A. P. (2022). Safety, tolerability and viral kinetics during SARS-CoV-2 human challenge. In Nature Medicine (Vol. 28, Issue 5). Nature Publishing Group. https://doi.org/10.21203/RS.3.RS-1121993/V1

![COVID-19 viral titres.png](COVID-19 viral titres.png)

Fig. 2 | Viral shedding after a short incubation period peaks rapidly after human SARS-CoV-2 challenge. Healthy adult volunteers were challenged intranasally with SARS-CoV-2. In the infected individuals (n = 18 biologically independent participants), VL in twice-daily nose and throat swab samples was measured by qPCR (blue) and FFA (red). ~~~ g, Median time to the last viral detection by FFA after inoculation is shown in red (*P = 0.0112)

FFA: viable virus measured by focus-forming assay (FFA).

*Note* from discusion 'Detailed viral kinetics show that some individuals still shed culturable virus at 12 days after inoculation (that is, up to 10 days after symptom onset), and, on average, viable virus was still detect- able 10 days after inoculation (up to 8 days after symptom onset)'.

Lets get a vector form of fig 2g.b


In [ ]:
import numpy as np
last_nose_ffa_value_counts  = {5.5:1, 7.5:1, 8:1, 8.5:2, 9:1, 9.5:2, 10:2, 10.5:4, 11.5:3, 12:1}
last_nose_ffa_value_days = []
for value, count in last_nose_ffa_value_counts.items():
    last_nose_ffa_value_days += [value] * count

last_nose_ffa_value_days = np.array(last_nose_ffa_value_days)
last_nose_ffa_value_days

In [ ]:
last_nose_ffa_rate_days = 1/last_nose_ffa_value_days
last_nose_ffa_rate_years = last_nose_ffa_rate_days*365.25
last_nose_ffa_rate_years

In [ ]:
import seaborn as sns
sns.histplot(last_nose_ffa_rate_years, bins= 30, kde=True)

In [ ]:
from scipy.stats import gamma
# see ?gamma.fit 
fit_a, fit_loc, fit_scale = gamma.fit(last_nose_ffa_rate_years)
gamma_fit = {'a': fit_a, 'loc': fit_loc, 'scale':fit_scale}
gamma_fit 

In [ ]:
from beast_pype.stat_distributions import gamma_df, plot_gamma_df
df = gamma_df(**gamma_fit)
df

In [ ]:
fig = plot_gamma_df(df)

**This does not allow for infection periods beyond 12 will have to choose something different.**

Lets try fixing location to 0.

In [ ]:
fit_a, fit_loc, fit_scale = gamma.fit(last_nose_ffa_rate_years, floc=0)
gamma_fit = {'a': fit_a, 'loc': fit_loc, 'scale':fit_scale}
df = gamma_df(**gamma_fit)
fig = plot_gamma_df(df)

Much better with a prior broad enough to account for changes seen in later strains.

In [ ]:
gamma_fit

In [ ]:
infectious_period_in_years = 10/365.25
rate_years = 1/infectious_period_in_years
rate_years

# Substitution Rate

In [ ]:
from src.beast_pype import gamma_from_mean
gamma_from_mean(mu=0.0004, sd=0.0003)